<a href="https://colab.research.google.com/github/abarrenos/Computational_Biology_UPM/blob/main/BigData/Exercise4_2_Challenge_predicting_insurance_risks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge: predicting insurance risks

### Inaccuracies in car insurance company’s claim predictions raise the cost of insurance for good drivers and reduce the price for bad ones. But Kaymo ™ wants to stop that!

### In this competition, you’re challenged to build a model that predicts the probability that a driver will initiate an auto insurance claim. While Kaymo ™ has used machine learning for the past 10 years, they’re looking to the CAIIS machine learning community to explore new, more powerful methods. A more accurate prediction will allow them to further tailor their prices, and hopefully make auto insurance coverage more accessible to more drivers.

## Setting up Spark

In [1]:
appname = "Logistic Regression - Documentation example"

# Look into https://spark.apache.org/downloads.html for the latest version
spark_mirror = "https://mirrors.sonic.net/apache/spark"
spark_version = "3.3.1"
hadoop_version = "3"

# Install Java 8 (Spark does not work with newer Java versions)
! apt-get update
! apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download and extract Spark binary distribution
! rm -rf spark-{spark_version}-bin-hadoop{hadoop_version}.tgz spark-{spark_version}-bin-hadoop{hadoop_version}
! wget -q {spark_mirror}/spark-{spark_version}/spark-{spark_version}-bin-hadoop{hadoop_version}.tgz
! tar xzf spark-{spark_version}-bin-hadoop{hadoop_version}.tgz

# The only 2 environment variables needed to set up Java and Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-{spark_version}-bin-hadoop{hadoop_version}"

# Set up the Spark environment based on the environment variable SPARK_HOME 
! pip install -q findspark
import findspark
findspark.init()

# Get the Spark session object (basic entry point for every operation)
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(appname).master("local[*]").getOrCreate()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://security.ubuntu.c

## Loading Database from Kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/kaggle"

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
# api.dataset_list(search="CAIIS Dogfood Day")
! rm -f caiis-dogfood-day-2020.zip
api.competition_download_files("caiis-dogfood-day-2020")

In [4]:
!rm *.csv

from zipfile import ZipFile
zf = ZipFile('caiis-dogfood-day-2020.zip')
zf.extractall('.')
zf.close()

rm: cannot remove '*.csv': No such file or directory


In [5]:
train = spark.read.format("csv").options(header=True, inferSchema=True).load("train.csv")
test = spark.read.format("csv").options(header=True, inferSchema=True).load("test.csv")

##Exploratory data analysis

In [6]:
print(train.count(),"rows x", len(train.columns), "columns")
train.show(10)

300000 rows x 32 columns
+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------+
| id|cat_0|cat_1|cat_2|cat_3|cat_4|cat_5|cat_6|cat_7|cat_8|cat_9|cat_10|cat_11|cat_12|cat_13|cat_14|cat_15|cat_16|cat_17|cat_18|             cont_0|             cont_1|             cont_2|             cont_3|             cont_4|             cont_5|             cont_6|             cont_7|             cont_8|             cont_9|           cont_10|target|
+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------

Each row in this dataset represents an insurance policy. You must predict the value for the target column, whether a policy resulted in a claim 1 or not 0 over a fixed time period. Variables prefaced with cat are categorical (nominal or ordinal, but not specified), while those prefaced with cont are continuous.

In [7]:
# Test data does not contain a target column.

print("Train columns:", train.columns,
      "\nTest columns:",test.columns)

Train columns: ['id', 'cat_0', 'cat_1', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cont_0', 'cont_1', 'cont_2', 'cont_3', 'cont_4', 'cont_5', 'cont_6', 'cont_7', 'cont_8', 'cont_9', 'cont_10', 'target'] 
Test columns: ['id', 'cat_0', 'cat_1', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cont_0', 'cont_1', 'cont_2', 'cont_3', 'cont_4', 'cont_5', 'cont_6', 'cont_7', 'cont_8', 'cont_9', 'cont_10']


In [8]:
print(train.printSchema())

root
 |-- id: integer (nullable = true)
 |-- cat_0: string (nullable = true)
 |-- cat_1: string (nullable = true)
 |-- cat_2: string (nullable = true)
 |-- cat_3: string (nullable = true)
 |-- cat_4: string (nullable = true)
 |-- cat_5: string (nullable = true)
 |-- cat_6: string (nullable = true)
 |-- cat_7: string (nullable = true)
 |-- cat_8: string (nullable = true)
 |-- cat_9: string (nullable = true)
 |-- cat_10: string (nullable = true)
 |-- cat_11: string (nullable = true)
 |-- cat_12: string (nullable = true)
 |-- cat_13: string (nullable = true)
 |-- cat_14: string (nullable = true)
 |-- cat_15: string (nullable = true)
 |-- cat_16: string (nullable = true)
 |-- cat_17: string (nullable = true)
 |-- cat_18: string (nullable = true)
 |-- cont_0: double (nullable = true)
 |-- cont_1: double (nullable = true)
 |-- cont_2: double (nullable = true)
 |-- cont_3: double (nullable = true)
 |-- cont_4: double (nullable = true)
 |-- cont_5: double (nullable = true)
 |-- cont_6: double 

I will first use statistical descriptors to get a first idea on the distribution of numerical features.

In [9]:
train.select(["cont_{}".format(i) for i in range(11)]).describe().show()

+-------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+-------------------+------------------+
|summary|              cont_0|             cont_1|             cont_2|              cont_3|             cont_4|              cont_5|              cont_6|             cont_7|              cont_8|             cont_9|           cont_10|
+-------+--------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+-------------------+------------------+
|  count|              300000|             300000|             300000|              300000|             300000|              300000|              300000|             300000|              300000|             300000|            300000|
|   mean|  0.5046677145265718|0.49429849784319463| 0.51666451783

In [10]:
# No null values are observed in our data
print("Null values:\n")
print({col: train.filter(train[col].isNull()).count() for col in train.columns})

Null values:

{'id': 0, 'cat_0': 0, 'cat_1': 0, 'cat_2': 0, 'cat_3': 0, 'cat_4': 0, 'cat_5': 0, 'cat_6': 0, 'cat_7': 0, 'cat_8': 0, 'cat_9': 0, 'cat_10': 0, 'cat_11': 0, 'cat_12': 0, 'cat_13': 0, 'cat_14': 0, 'cat_15': 0, 'cat_16': 0, 'cat_17': 0, 'cat_18': 0, 'cont_0': 0, 'cont_1': 0, 'cont_2': 0, 'cont_3': 0, 'cont_4': 0, 'cont_5': 0, 'cont_6': 0, 'cont_7': 0, 'cont_8': 0, 'cont_9': 0, 'cont_10': 0, 'target': 0}


##Preprocessing of data

In order to build an accurate predictive model, we should take into accound all variables that might influence the probability of insurance claim.

I would like to consider the categorical features of the dataframe. First I need to convert these categorical "string" values into indexes using String Indexer. I will consider these features as non-categorical, therefore, One-Hot Encoding of the features must be performed.

In [11]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.pipeline import Pipeline

cat_features = ["cat_{}".format(col) for col in range(19)]
cat_idx = ["cat_{}_idx".format(col) for col in range(19)]
cat_OneHot = ["cat_{}_OneHot".format(col) for col in range(19)]

indexer = StringIndexer(handleInvalid="skip", 
                        inputCols=cat_features,
                        outputCols=cat_idx).fit(train)
                        
train2 = indexer.transform(train)

encoder = OneHotEncoder(dropLast = False, handleInvalid = "error",
                    inputCols = cat_idx,
                    outputCols = cat_OneHot).fit(train2)

train_transformed = encoder.transform(train2)[cat_OneHot+
                                        ["cont_{}".format(col) for col in range(11)]+
                                        ["target"]]
train_transformed.show(10)

+-------------+--------------+--------------+--------------+---------------+--------------+--------------+---------------+---------------+--------------+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------+
| cat_0_OneHot|  cat_1_OneHot|  cat_2_OneHot|  cat_3_OneHot|   cat_4_OneHot|  cat_5_OneHot|  cat_6_OneHot|   cat_7_OneHot|   cat_8_OneHot|  cat_9_OneHot|   cat_10_OneHot|cat_11_OneHot|cat_12_OneHot|cat_13_OneHot|cat_14_OneHot|cat_15_OneHot|cat_16_OneHot|cat_17_OneHot|cat_18_OneHot|             cont_0|             cont_1|             cont_2|             cont_3|             cont_4|             cont_5|             cont_6|             cont_7|             cont_8|             cont_9|         

Lastly, we assemble all considered features into a single vector using Vector Assembler.

In [22]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(handleInvalid = "skip",
                            inputCols = train_transformed.columns[:-1],
                            outputCol = "features")
print("Train data:")
train_assembled = assembler.transform(train_transformed)
train_assembled.select(["features", "target"]).show(10)

Train data:
+--------------------+------+
|            features|target|
+--------------------+------+
|(632,[0,2,17,37,6...|     1|
|(632,[0,2,17,36,4...|     0|
|(632,[0,4,17,36,4...|     1|
|(632,[0,2,20,37,4...|     0|
|(632,[0,3,17,36,5...|     0|
|(632,[0,6,18,36,4...|     0|
|(632,[0,2,18,36,4...|     0|
|(632,[0,7,18,36,5...|     1|
|(632,[0,10,20,36,...|     0|
|(632,[0,3,17,36,5...|     0|
+--------------------+------+
only showing top 10 rows



Since test data does not contain a target feature (unlabelled data) to assess the accuracy of our model, we need to split the training data in order to generate a validation data set.

Besides this, I will assemble the different data transformers into a single Pipeline to facilitate the preprocessing of test data.

In [24]:
from pyspark.ml.pipeline import Pipeline

final_train, final_valid = train_assembled.randomSplit([0.8,0.2], seed = 5)

pipeline = Pipeline(stages = [indexer, encoder, assembler]).fit(train)
final_test = pipeline.transform(test)

print("Train data:", final_train.count())
print(final_train.select(["features", "target"]).show(10))

print("Validation data:", final_valid.count())
print(final_valid.select(["features", "target"]).show(10))

print("Test data:", final_test.count())
print(final_test.select(["features"]).show(10))

Train data: 240033
+--------------------+------+
|            features|target|
+--------------------+------+
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     1|
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     1|
|(632,[0,2,17,36,4...|     1|
|(632,[0,2,17,36,4...|     1|
|(632,[0,2,17,36,4...|     0|
+--------------------+------+
only showing top 10 rows

None
Validation data: 59967
+--------------------+------+
|            features|target|
+--------------------+------+
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     0|
|(632,[0,2,17,36,4...|     1|
|(632,[0,2,17,36,4...|     0|
+--------------------+------+
only showing top 10 rows

None
Test data: 199986
+--------------------+
|            fe

## Building our model

We train different tree modes with the training data to predict the target categories of the test data.

In [25]:
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier, DecisionTreeClassifier)

dtc_model = DecisionTreeClassifier(featuresCol="features", labelCol="target").fit(final_train)
rfc_model = RandomForestClassifier(numTrees = 100, featuresCol="features", labelCol="target").fit(final_train)
gbt_model = GBTClassifier(featuresCol="features", labelCol="target").fit(final_train)


Predict target features for validation data using fitted models.

In [38]:
dtc_pred = dtc_model.transform(final_valid).select(["features", "target", "rawPrediction", "probability", "prediction"])
rfc_pred = rfc_model.transform(final_valid).select(["features", "target", "rawPrediction", "probability", "prediction"])
gbt_pred = gbt_model.transform(final_valid).select(["features", "target", "rawPrediction", "probability", "prediction"])

dtc_pred.show(5)
rfc_pred.show(5)
gbt_pred.show(5)

+--------------------+------+-----------------+--------------------+----------+
|            features|target|    rawPrediction|         probability|prediction|
+--------------------+------+-----------------+--------------------+----------+
|(632,[0,2,17,36,4...|     0|[63221.0,24326.0]|[0.72213782311215...|       0.0|
|(632,[0,2,17,36,4...|     0|  [23532.0,846.0]|[0.96529657888259...|       0.0|
|(632,[0,2,17,36,4...|     0|  [23532.0,846.0]|[0.96529657888259...|       0.0|
|(632,[0,2,17,36,4...|     0| [54408.0,7921.0]|[0.87291629899404...|       0.0|
|(632,[0,2,17,36,4...|     0| [54408.0,7921.0]|[0.87291629899404...|       0.0|
+--------------------+------+-----------------+--------------------+----------+
only showing top 5 rows

+--------------------+------+--------------------+--------------------+----------+
|            features|target|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|(632,

Since training the model took a long time because the models were fitted with a relatively large amount of data, I decided to train the data once and store the prediction results as a csv in my Google Drive. I was unable to save the vector-assembled features, but features with single values and final predictions were correctly saved.

In [39]:
''' Prediction data is saved in Drive '''

# dtc_pred.select(dtc_pred.columns[:-23] + ["prediction"]).write.csv(path = "./drive/MyDrive/files/dtc_pred.csv", mode = "overwrite")
# rfc_pred.select(rfc_pred.columns[:-23] + ["prediction"]).write.csv(path = "./drive/MyDrive/files/rfc_pred.csv", mode = "overwrite")
# gbt_pred.select(gbt_pred.columns[:-23] + ["prediction"]).write.csv(path = "./drive/MyDrive/files/gbt_pred.csv", mode = "overwrite")

''' Prediction data is loaded from Drive '''

# dtc_pred = spark.read.format("csv").load("./drive/MyDrive/files/dtc_pred.csv/*")  
# rfc_pred = spark.read.format("csv").load("./drive/MyDrive/files/rfc_pred.csv/*")  
# gbt_pred = spark.read.format("csv").load("./drive/MyDrive/files/gbt_pred.csv/*")  


' Prediction data is loaded from Drive '

I identified the most important features for each model, and the differ between them.

In [40]:
# Identify the relative importances of every feature

for model in [dtc_model, rfc_model, gbt_model]:
  max_idx = list(model.featureImportances.values).index(model.featureImportances.values.max())
  important_feature = model.featureImportances.indices[max_idx]
  print("\n"+str(model))
  print("feature:", important_feature, "\tMax_importance:", model.featureImportances.values.max())




DecisionTreeClassificationModel: uid=DecisionTreeClassifier_74d12f4b2d56, depth=5, numNodes=31, numClasses=2, numFeatures=632
feature: 610 	Max_importance: 0.5013519487767588

RandomForestClassificationModel: uid=RandomForestClassifier_a420269506f7, numTrees=100, numClasses=2, numFeatures=632
feature: 304 	Max_importance: 0.10264472726519136

GBTClassificationModel: uid = GBTClassifier_c16de23716f1, numTrees=20, numClasses=2, numFeatures=632
feature: 606 	Max_importance: 0.11550300146775486


Now, I will evaluate the model predictive accuracy on validation data using Binary and Multiclass Classificator Evaluators.


In [49]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

binevaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='target')
mcevaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='target', probabilityCol="probability")

'labelCol: label column name. (default: label, current: target)\nmetricName: metric name in evaluation (areaUnderROC|areaUnderPR) (default: areaUnderROC)\nnumBins: Number of bins to down-sample the curves (ROC curve, PR curve) in area computation. If 0, no down-sampling will occur. Must be >= 0. (default: 1000)\nrawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction, current: rawPrediction)\nweightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)'

In [50]:
for pred in [dtc_pred, rfc_pred, gbt_pred]:
  print("\nModel:", str(model))
  print("F1:", round(mcevaluator.evaluate(pred, params = {mcevaluator.metricName: "accuracy"}), 2))
  print("Accuracy:", round(mcevaluator.evaluate(pred, params = {mcevaluator.metricName: "accuracy"}), 2))
  print("weightedPrecision:", round(mcevaluator.evaluate(pred, params = {mcevaluator.metricName: "accuracy"}), 2))
  print("Area Under ROC:", round(binevaluator.evaluate(pred, params = {mcevaluator.metricName: "accuracy"}), 2))



Model: GBTClassificationModel: uid = GBTClassifier_c16de23716f1, numTrees=20, numClasses=2, numFeatures=632
F1: 0.82
Accuracy: 0.82
weightedPrecision: 0.82
Area Under ROC: 0.66

Model: GBTClassificationModel: uid = GBTClassifier_c16de23716f1, numTrees=20, numClasses=2, numFeatures=632
F1: 0.82
Accuracy: 0.82
weightedPrecision: 0.82
Area Under ROC: 0.78

Model: GBTClassificationModel: uid = GBTClassifier_c16de23716f1, numTrees=20, numClasses=2, numFeatures=632
F1: 0.83
Accuracy: 0.83
weightedPrecision: 0.83
Area Under ROC: 0.8


Using this models, we can predict the target feature labels on our test data.

In [51]:
dtc_pred = dtc_model.transform(final_test).select(["features", "rawPrediction", "probability", "prediction"])
rfc_pred = rfc_model.transform(final_test).select(["features", "rawPrediction", "probability", "prediction"])
gbt_pred = gbt_model.transform(final_test).select(["features", "rawPrediction", "probability", "prediction"])

dtc_pred.show(5)
rfc_pred.show(5)
gbt_pred.show(5)

+--------------------+----------------+--------------------+----------+
|            features|   rawPrediction|         probability|prediction|
+--------------------+----------------+--------------------+----------+
|(632,[0,4,17,38,4...|[54408.0,7921.0]|[0.87291629899404...|       0.0|
|(632,[1,5,18,36,5...|[54408.0,7921.0]|[0.87291629899404...|       0.0|
|(632,[0,3,17,37,5...|[54408.0,7921.0]|[0.87291629899404...|       0.0|
|(632,[1,2,17,36,4...| [8404.0,2571.0]|[0.76574031890660...|       0.0|
|(632,[0,5,17,37,4...| [23524.0,142.0]|[0.99399983098115...|       0.0|
+--------------------+----------------+--------------------+----------+
only showing top 5 rows

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(632,[0,4,17,38,4...|[82.6907643551771...|[0.82690764355177...|       0.0|
|(632,[1,5,18,36,5...|[